In [1]:
import pickle
from utils.build_graph import build_graph_dgl,build_graph_pyg
import torch
from utils.utils import (set_random_seed)
import argparse
import logging
import os
import copy
import datetime
import os
import pickle
import warnings
from lp_models.MetaTransformer import ExplainableGraphTransformer,RGCNClassfier
import numpy as np
import scipy.sparse as sp
import torch
from sklearn.metrics import roc_auc_score
import torch.optim as optim
from utils.io import load_data
import tqdm
import copy
import datetime
import gc
import itertools
import os
import pickle
import queue
import random
import time
import warnings
from dataclasses import dataclass

import dgl
import dgl.nn as dglnn
import dgl.sparse as dglsp
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import torch as th
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from dgl import shortest_dist
from dgl.base import NID
from dgl.convert import to_heterogeneous, to_homogeneous
from dgl.data import AsGraphPredDataset
from dgl.data.utils import save_graphs
from dgl.dataloading import GraphDataLoader
from dgl.nn import HeteroEmbedding, PathEncoder
from dgl.random import choice
from dgl.sampling import random_walk
from ogb.graphproppred import DglGraphPropPredDataset, Evaluator, collate_dgl
from ogb.graphproppred.mol_encoder import AtomEncoder
from sklearn.model_selection import train_test_split
from torch import Tensor, nn
from torch.nn import init
from torch.nn.parallel import DistributedDataParallel
from torch.optim import SparseAdam
from torch.utils import data
from torch.utils.data import DataLoader
from tqdm import tqdm

from utils.build_graph import build_graph_dgl
def readFile(filePath):
    with open(f"./dataset/{dataset}/{filePath}.pkl", 'rb') as file:
        return pickle.loads(file.read())

In [2]:
dataset='quora'
Answers_train=readFile('Answers_train')
Answers_dev=readFile('Answers_dev')
Answers_test=readFile('Answers_test')
Questions=readFile('Questions')
Users=readFile('Users')

In [3]:
with open(f"./dataset/quora_cache/Answers_test_dataset.pkl", 'rb') as file:
    Answers_test_dataset= pickle.loads(file.read())

In [4]:
 with open(f"./dataset/quora_cache/g.pkl", 'rb') as f:
    g = pickle.load(f)

In [5]:
import copy
import datetime
import gc
import itertools
import os
import pickle
import queue
import random
import time
import warnings
from dataclasses import dataclass

import dgl
import dgl.nn as dglnn
import dgl.sparse as dglsp
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import torch as th
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from dgl import shortest_dist
from dgl.base import NID
from dgl.convert import to_heterogeneous, to_homogeneous
from dgl.data import AsGraphPredDataset
from dgl.data.utils import save_graphs
from dgl.dataloading import GraphDataLoader
from dgl.nn import HeteroEmbedding, PathEncoder
from dgl.random import choice
from dgl.sampling import random_walk
from ogb.graphproppred import DglGraphPropPredDataset, Evaluator, collate_dgl
from ogb.graphproppred.mol_encoder import AtomEncoder
from sklearn.model_selection import train_test_split
from torch import Tensor, nn
from torch.nn import init
from torch.nn.parallel import DistributedDataParallel
from torch.optim import SparseAdam
from torch.utils import data
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy
import datetime
import gc
import itertools
import os
import pickle
import queue
import random
import time
import warnings
from dataclasses import dataclass

import dgl
import dgl.nn as dglnn
import dgl.sparse as dglsp
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import torch as th
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from dgl import shortest_dist
from dgl.base import NID
from dgl.convert import to_heterogeneous, to_homogeneous
from dgl.data import AsGraphPredDataset
from dgl.data.utils import save_graphs
from dgl.dataloading import GraphDataLoader
from dgl.nn import HeteroEmbedding, PathEncoder
from dgl.random import choice
from dgl.sampling import random_walk
from ogb.graphproppred import DglGraphPropPredDataset, Evaluator, collate_dgl
from ogb.graphproppred.mol_encoder import AtomEncoder
from sklearn.model_selection import train_test_split
from torch import Tensor, nn
from torch.nn import init
from torch.nn.parallel import DistributedDataParallel
from torch.optim import SparseAdam
from torch.utils import data
from torch.utils.data import DataLoader
from tqdm import tqdm

from utils.build_graph import build_graph_dgl

from utils.build_graph import build_graph_dgl

os.environ['DGLBACKEND'] = 'pytorch'
warnings.filterwarnings('ignore')



class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()
        # 实例化HeteroGraphConv，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregate是聚合函数的类型
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # 输入是节点的特征字典
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h


class MetaPathTransformer(nn.Module):
    def __init__(self, max_length, num_head, embedding_size, device):
        super().__init__()
        self.position_embeddings = nn.Embedding(max_length, embedding_size)
        self.nodetype_embeddings = nn.Embedding(4, embedding_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_size, batch_first=True, nhead=num_head)
        self.position_embeddings_index = torch.tensor(list(range(max_length))).to(device)
        self.max_length = max_length
        self.embedding_size = embedding_size
        self.device = device

    def forward(self, node_embeddings, node_types_link, src_mask):
        embedding = node_embeddings + \
            self.position_embeddings(self.position_embeddings_index)+self.nodetype_embeddings(node_types_link)
        embedding = embedding.view(-1, self.max_length, self.embedding_size)
        return self.encoder_layer(embedding, src_key_padding_mask=src_mask.view(-1, self.max_length))


class DotProductSimilarity(nn.Module):
    def __init__(self):
        super(DotProductSimilarity, self).__init__()

    def forward(self, tensor_1, tensor_2):
        result = (tensor_1 * tensor_2).sum(dim=-1)
        return result


class PathGather(nn.Module):
    def __init__(self, embedding_size=128, max_paths=128):
        super(PathGather, self).__init__()
        self.dot = DotProductSimilarity()
        self.embedding_size = embedding_size
        self.max_paths = max_paths

    def forward(self, link_user_embeddings, link_answer_embeddings):
        score = self.dot(link_user_embeddings, link_answer_embeddings).view(-1, self.max_paths)
        attention = F.softmax(score)
        attention = attention.unsqueeze(-1)
        link_user_path_embeddings = torch.sum(
            attention*(link_user_embeddings.view(-1, self.max_paths, self.embedding_size)), 1)
        link_answer_path_embeddings = torch.sum(
            attention*(link_answer_embeddings.view(-1, self.max_paths, self.embedding_size)), 1)
        return self.dot(link_user_path_embeddings, link_answer_path_embeddings),attention


class ExplainableGraphTransformer(nn.Module):
    def __init__(self, embeddingDict, embedding_size, etypes, max_length, num_head, device='cuda:0', max_paths=10):
        super().__init__()
        self.embeddings = HeteroEmbedding(embeddingDict, embedding_size)
        self.rgcn = RGCN(embedding_size, embedding_size, embedding_size, etypes)
        self.metapathencoder = MetaPathTransformer(max_length, num_head, embedding_size, device)
        self.pathgather = PathGather(embedding_size, max_paths)

        self.device = device
        self.id2nodetype={0:'user',1:'answer',2:'topic',3:'question'}
    def forward(self, g, node_paths_link, type_paths_link, masks_link, answer_position_link):
        # node_types=[[nodetype2id[nodetype] for nodetype in path] for path in type_paths]
        h = self.embeddings({key: g.nodes(key) for key in g.ntypes})
        x = self.rgcn(g, h)
        node_embeddings = torch.stack([torch.stack([torch.stack([x[self.id2nodetype[key]][value] for value, key in zip(node_path, type_path)])
                                      for node_path, type_path in zip(node_paths, type_paths)]) for node_paths, type_paths in zip(node_paths_link, type_paths_link.tolist())])
        # node_types_link=torch.tensor([[[nodetype2id[nodetype] for nodetype in path] for path in type_paths] for type_paths in type_paths_link])
        link_path_embeddings = self.metapathencoder(node_embeddings.to(
            self.device), type_paths_link.to(self.device), masks_link.to(self.device))
        index = answer_position_link.view(-1)



        #-----------------------消融实验----------------------------------
        link_user_embeddings = link_path_embeddings[:, 0, :]
        link_answer_embeddings = torch.stack([link_path_embeddings[num, i, :]
                                             for num, i in zip(range(len(index)), index)])
        return self.pathgather(link_user_embeddings, link_answer_embeddings)

class RGCNClassfier(nn.Module):
    def __init__(self, embeddingDict, embedding_size, etypes,max_length=5,max_paths=10):
        super().__init__()
        self.embeddings = HeteroEmbedding(embeddingDict, embedding_size)
        self.rgcn = RGCN(embedding_size, embedding_size, embedding_size, etypes)
        self.dot=DotProductSimilarity()
        self.max_paths=max_paths
        self.max_length=max_length
        self.embedding_size=embedding_size
        self.id2nodetype={0:'user',1:'answer',2:'topic',3:'question'}
    def forward(self, g, node_paths_link, type_paths_link, masks_link, answer_position_link):
        # node_types=[[nodetype2id[nodetype] for nodetype in path] for path in type_paths]
        h = self.embeddings({key: g.nodes(key) for key in g.ntypes})
        x = self.rgcn(g, h)
        link_path_embeddings = torch.stack([torch.stack([torch.stack([x[self.id2nodetype[key]][value] for value, key in zip(node_path, type_path)])
                                      for node_path, type_path in zip(node_paths, type_paths)]) for node_paths, type_paths in zip(node_paths_link, type_paths_link.tolist())])
        link_path_embeddings=link_path_embeddings.view(-1, self.max_length, self.embedding_size)
        index = answer_position_link.view(-1)
        link_user_embeddings = link_path_embeddings[:, 0, :]
        link_answer_embeddings = torch.stack([link_path_embeddings[num, i, :]
                                             for num, i in zip(range(len(index)), index)])
        score = self.dot(link_user_embeddings, link_answer_embeddings).view(-1, self.max_paths)
        return score[:,0]




In [6]:
device='cpu' 
embeddingDict={nodeType:g.num_nodes(nodeType) for nodeType in g.ntypes}
model = ExplainableGraphTransformer(embeddingDict,128, g.etypes,max_length=5,num_head=8,device=device,max_paths=10)
model.to(device)
g=g.to(device)


In [7]:
#'outputs/best_mrr_model_quora_backup.pt'

model.load_state_dict(torch.load('outputs/best_mrr_model_quora_backup.pt'))

<All keys matched successfully>

In [8]:
Answers_test.head(3)

,answerId,questionId,askerUid,answerText,questionTitle,questionUrl,answerNumUpvotes,answerNumSharers,answerCreationTime,answerNumViews,answerNumApprovedShares,answerNumDisplayComments,questionTopics,numPublicAnswers,questionTopicsNames,users,labels
0,152932,229683,488864,You know that feeling when you're scrolling th...,Where can I find good content for my blogs?,/Where-can-I-find-good-content-for-my-blogs,0,0,2021-12-30 18:59:38.643311,62,0,0,"[16103, 9974, 418, 1603, 2302]",340,"[Content Marketing, Content Strategy, Blogs, B...","[1659, 2807, 2676, 3954, 566]","[1, 0, 0, 0, 0]"
1,95565,213720,162124,Not if they want to continue to exist as a nat...,Will Russia invade the Baltics?,https://ondiplomacywarfare.quora.com/Will-Russ...,5,0,2022-05-19 03:39:36.918216,552,0,1,"[25857, 54045, 44849, 65707, 3117, 4959]",426,"[Baltic States, Invasions, Foreign Policy of R...","[1853, 2765, 2742, 1385, 2009]","[1, 0, 0, 0, 0]"
2,116495,56331,527428,"Based on descriptions of an INTP personality, ...",Which field in computer science is best for an...,/Which-field-in-computer-science-is-best-for-a...,0,0,2022-10-18 02:58:37.041072,63,0,0,"[56620, 58555, 45654, 39973, 6868, 13292, 2731...",428,"[Career Advice and Computer Science Education,...","[1435, 1093, 2621, 421, 3232]","[1, 0, 0, 0, 0]"


In [9]:
score,attention=model(g,*Answers_test_dataset[10:12])

In [10]:
attention[0].cpu().detach().numpy().tolist()

[[0.0018483410822227597],
 [0.47560933232307434],
 [0.02942093461751938],
 [3.6818259104620665e-05],
 [0.35814154148101807],
 [0.12175766378641129],
 [0.00922316312789917],
 [0.00043381043360568583],
 [0.0032571330666542053],
 [0.0002713028807193041]]

In [12]:
Users[Users.uid==1435]

,uid,numPublicAnswers,numProfileQuestions,postsCount,followerCount,followingCount,numFollowedTribes,numFollowedTopics,numFollowedQuestions,profileUrl,followings,followers,userName,followedQuestions,followedTopics,askedQuestions,writtenAnswers
2745,1435,425,1,89,15,0,26,48,12,/profile/William-Hembree-3,[],"[141, 1286, 4620, 2776, 997, 3059, 3120, 3065,...",William-Hembree-3,"[221586, 197966, 21746, 213510, 45865, 90958, ...","[13292, 332, 1924, 7151, 500, 1647, 911, 4959,...",[16354],"[116608, 116497, 116646, 116623, 116593, 11665..."


In [14]:
Topics=readFile('Topics')

In [16]:
Topics[Topics.tid==13292]

,tid,topicName
4649,13292,Computer Science Education


In [19]:
Answers_test.head(3)

,answerId,questionId,askerUid,answerText,questionTitle,questionUrl,answerNumUpvotes,answerNumSharers,answerCreationTime,answerNumViews,answerNumApprovedShares,answerNumDisplayComments,questionTopics,numPublicAnswers,questionTopicsNames,users,labels
0,152932,229683,488864,You know that feeling when you're scrolling th...,Where can I find good content for my blogs?,/Where-can-I-find-good-content-for-my-blogs,0,0,2021-12-30 18:59:38.643311,62,0,0,"[16103, 9974, 418, 1603, 2302]",340,"[Content Marketing, Content Strategy, Blogs, B...","[1659, 2807, 2676, 3954, 566]","[1, 0, 0, 0, 0]"
1,95565,213720,162124,Not if they want to continue to exist as a nat...,Will Russia invade the Baltics?,https://ondiplomacywarfare.quora.com/Will-Russ...,5,0,2022-05-19 03:39:36.918216,552,0,1,"[25857, 54045, 44849, 65707, 3117, 4959]",426,"[Baltic States, Invasions, Foreign Policy of R...","[1853, 2765, 2742, 1385, 2009]","[1, 0, 0, 0, 0]"
2,116495,56331,527428,"Based on descriptions of an INTP personality, ...",Which field in computer science is best for an...,/Which-field-in-computer-science-is-best-for-a...,0,0,2022-10-18 02:58:37.041072,63,0,0,"[56620, 58555, 45654, 39973, 6868, 13292, 2731...",428,"[Career Advice and Computer Science Education,...","[1435, 1093, 2621, 421, 3232]","[1, 0, 0, 0, 0]"


In [24]:
Answers_test.loc[2].answerText

'Based on descriptions of an INTP personality, you’d probably feel at home as a software developer. I’d suggest avoiding user interface design but back end (aka server) or embedded roles would seem to be good choices. I’d suggest avoiding IT roles, however, and work on a company’s products instead. A2A: “ Which field in computer science is best for an INTP as a career? ”'

In [25]:
Answers_test.loc[2].questionTitle

'Which field in computer science is best for an INTP as a career?'

In [11]:
[  1435,  13292,  56331, 116495,      0]
[  1435, 261256,  13292,  56331, 116495]
[  1435,  17579,  13292,  56331, 116495]

(tensor([[  1435, 116495,      0,      0,      0],
         [  1435,  13292,  56331, 116495,      0],
         [  1435, 116495,      0,      0,      0],
         [  1435, 116495,      0,      0,      0],
         [  1435, 261256,  13292,  56331, 116495],
         [  1435,  17579,  13292,  56331, 116495],
         [  1435,  17579,  39973,  56331, 116495],
         [  1435, 116495,      0,      0,      0],
         [  1435, 116495,      0,      0,      0],
         [  1435, 116495,      0,      0,      0]]),
 tensor([[0, 1, 1, 1, 1],
         [0, 2, 3, 1, 1],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 3, 2, 3, 1],
         [0, 3, 2, 3, 1],
         [0, 3, 2, 3, 1],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0]]),
 tensor([[False, False,  True,  True,  True],
         [False, False, False, False,  True],
         [False, False,  True,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False, False, Fals

In [ ]:
Now the Russian Government as headed by Putin have decided

If you had any doubt what the US policy is toward the

 any different towards her, but it was already to late, trust had been broken. To make matters worse she admitted that she had gone to see her ex that she is being protected from by SPAN a battered women’s safe house. SPAN 